In [ ]:
import minsearch
import json
import openai
from openai import OpenAI

In [ ]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)
        
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)
index.fit(documents)

In [ ]:
q = "can i still enroll?"

In [ ]:
boost = { "question" : 3, "section" : 0.5 }

results = index.search(
    query = q,
    filter_dict = {"course": "data-engineering-zoomcamp"},
    boost_dict = boost,
    num_results=5
)

context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

context

In [ ]:
prompt_template = """
You're a course teaching assistant. 
Answer the QUESTION based on the CONTEXT. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output IDK
Answer in at most three sentences.

QUESTION: {question}

CONTEXT: 
{context}
"""

prompt = prompt_template.format(question=q, context=context).strip()
print(prompt)

In [ ]:
client = OpenAI()

In [ ]:
response = client.chat.completions.create(
    model = 'gpt-4o',
    messages = [{"role":"user", "content":prompt}]
)

In [ ]:
response.choices[0].message.content